# Interactive Recommendation System with Word Embeddings using Word2Vec, Plotly, and NetworkX

## Project Breakdown
- Task 1: Introduction
- Task 2: Exploratory Data Analysis and Preprocessing (you are here)
- Task 3: Word2Vec with Gensim
- Task 4: Exploring Results
- Task 5: Building and Visualizing Interactive Network Graph

## Task 2: Exploratory Data Analysis and Preprocessing

[This is the dataset](https://eightportions.com/datasets/Recipes/#fn:1) we will be using. It is collated by Ryan Lee, sourced from [Food Network](https://www.foodnetwork.com/), [Epicurious](https://www.epicurious.com/), and [Allrecipes](https://www.allrecipes.com/).

In [1]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm
import pandas as pd
import pickle
import string
import json

In [2]:
recipe_sources = ['ar', 'epi', 'fn']

In [3]:
df = pd.DataFrame()
sources, titles, ingredients, instructions = [], [], [], []
for recipe_source in recipe_sources:
    data = json.load(open(f'Data/Dataset/recipes_raw_nosource_{recipe_source}.json', 'r'))
    for _, recipe in data.items():
        if ('title' in recipe) and ('ingredients' in recipe) and ('instructions' in recipe):
            # append to a list of the source
            sources.append(recipe_source)
            # append to a list of the titles
            titles.append(recipe['title'])
            # append to a list of a list of ingredients, removing the word ADVERTISEMENT
            ingredients.append([ingredient_line.replace('ADVERTISEMENT', '') for ingredient_line in recipe['ingredients']])
            # append to a silst of instructions, remmoving the word ADVERTISEMENT and replace \n with space characters
            new_instruction = recipe.get('instructions', '')
            
            if new_instruction:
                new_instruction.replace('ADVERTISEMENT', '')
                new_instruction.replace('\n', ' ')
                
            instructions.append(new_instruction)
            
df['source'] = sources
df['title'] = titles
df['ingredients'] = ingredients
df['instructions'] = instructions

In [4]:
df.head()

,source,title,ingredients,instructions
0,ar,Slow Cooker Chicken and Dumplings,"[4 skinless, boneless chicken breast halves , ...","Place the chicken, butter, soup, and onion in ..."
1,ar,Awesome Slow Cooker Pot Roast,[2 (10.75 ounce) cans condensed cream of mushr...,"In a slow cooker, mix cream of mushroom soup, ..."
2,ar,Brown Sugar Meatloaf,"[1/2 cup packed brown sugar , 1/2 cup ketchup ...",Preheat oven to 350 degrees F (175 degrees C)....
3,ar,Best Chocolate Chip Cookies,"[1 cup butter, softened , 1 cup white sugar , ...",Preheat oven to 350 degrees F (175 degrees C)....
4,ar,Homemade Mac and Cheese Casserole,"[8 ounces whole wheat rotini pasta , 3 cups fr...",Preheat oven to 350 degrees F. Line a 2-quart ...


In [5]:
df.dropna(inplace=True)

In [6]:
df.isna().sum()

source          0
title           0
ingredients     0
instructions    0
dtype: int64

In [7]:
to_remove = [
    'tablespoon',
    'tablespoons',
    'teaspoon',
    'teaspoons',
    'tsp',
    'tsps',
    'tbsp',
    'tbsps',
    'pound',
    'pounds',
    'grams',
    'mg',
    'ounce'
    'ounces',
    'kg',
    'crushed',
    'chopped',
    'finely',
    'softened',
    'cups',
    'cup'
]

In [8]:
translation_table = str.maketrans('','',string.punctuation + string.digits)

In [9]:
stopwords = ["on", "this"]
str1 = "how about on for this time thanks"
str1 = remove_stopwords(str1, stopwords)
str1

'how about for time thanks'

In [10]:
def preprocess(items):
    # fill me in!
    res = []
    
    for item in items:
        temp = item.lower().replace('-', ' ')
        temp = temp.translate(translation_table)
        temp = remove_stopwords(temp, to_remove)
        res.append(temp.split())
        
    return res

In [11]:
instructions = df['instructions'].to_list()
ingredients = [','.join(s) for s in df['ingredients'].to_list()]

In [12]:
instructions[0], ingredients[0]

('Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover.\nCover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.\n',
 '4 skinless, boneless chicken breast halves ,2 tablespoons butter ,2 (10.75 ounce) cans condensed cream of chicken soup ,1 onion, finely diced ,2 (10 ounce) packages refrigerated biscuit dough, torn into pieces ,')

In [13]:
train_data = preprocess(instructions + ingredients)

In [14]:
train_data[0]

['place',
 'the',
 'chicken',
 'butter',
 'soup',
 'and',
 'onion',
 'in',
 'a',
 'slow',
 'cooker',
 'and',
 'fill',
 'with',
 'enough',
 'water',
 'to',
 'cover',
 'cover',
 'and',
 'cook',
 'for',
 'to',
 'hours',
 'on',
 'high',
 'about',
 'minutes',
 'before',
 'serving',
 'place',
 'the',
 'torn',
 'biscuit',
 'dough',
 'in',
 'the',
 'slow',
 'cooker',
 'cook',
 'until',
 'the',
 'dough',
 'is',
 'no',
 'longer',
 'raw',
 'in',
 'the',
 'center']

In [15]:
with open('Data/train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

: 